In [6]:
import os
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')
import csv
import numpy as np
import scipy.stats as stats

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
lists_directories = ['/content/drive/MyDrive/Data/prepilot/fitbit','/content/drive/MyDrive/Data/pilot/fitbit/control','/content/drive/MyDrive/Data/pilot/fitbit/intervention']
ext = ('.csv')

writer =  pd.ExcelWriter('/content/drive/MyDrive/Data/ANOVA.xlsx') 

for cnt,fitbit_data_dir in enumerate(lists_directories):
    # open the file in the write mode
    if (cnt == 0):
        print("PREPILOT\n")
    elif (cnt == 1):
        print("PILOT_CONTROL\n")
    elif (cnt == 2):
        print("PILOT_INTERVENTION\n")

    # Read fitbit_data
    fitbit_data_lst = []
    for files in os.listdir(fitbit_data_dir):
        if files.endswith(ext):
            ##print(fitbit_data_dir + "/" + files)
            fitbit_data = pd.read_csv(fitbit_data_dir + "/" + files, delimiter=',', dtype=object)
            #fitbit_data['pid'] = files[0:-4]
            print("==========================")
            print(files[0:-4])
            print("==========================")

            fitbit_data.rename(columns={'date': 'timestamp'}, inplace=True)
            try:
                fitbit_data = fitbit_data[['timestamp', 'steps']]
            except:
                continue

            fitbit_data['timestamp'] = pd.to_datetime(fitbit_data['timestamp'])
            fitbit_data['Year'] = fitbit_data['timestamp'].dt.year
            fitbit_data['Week'] = fitbit_data['timestamp'].dt.week
            fitbit_data = fitbit_data.dropna()
            fitbit_data['steps'] = fitbit_data['steps'].astype(int, errors='ignore')
            
            #print(fitbit_data.dtypes)
            #fitbit_data = fitbit_data.groupby(by=["Year","Week"], as_index=False, sort=True).sum()
            fitbit_data = fitbit_data.sort_values(by=["Year",'Week'])
            numOfWeeks = fitbit_data.shape[0]
            #print(fitbit_data)
            #fitbit_data['Week'] = pd.Series(list(range(1,numOfWeeks+1)))
            fitbit_data = fitbit_data.drop('Year', 1)

            fitbit_data['Week'] = fitbit_data['Week'].astype(str)
            fitbit_data['Week'] = fitbit_data['Week'].map('Week {}'.format)
            #fitbit_data = fitbit_data.sort_values(by=['Week'])

            #print(fitbit_data)

            #ANOVA TESTING

            # Create ANOVA backbone table
            data = [['Between Groups', '', '', '', '', '', ''], ['Within Groups', '', '', '', '', '', ''],
                    ['Total', '', '', '', '', '', '']]
            anova_table = pd.DataFrame(data, columns=['Source of Variation', 'SS', 'df', 'MS', 'F', 'P-value', 'F crit'])
            anova_table.set_index('Source of Variation', inplace=True)

            # calculate SSTR and update anova table
            x_bar = fitbit_data['steps'].mean()
            SSTR = fitbit_data.groupby('Week').count() * (fitbit_data.groupby('Week').mean() - x_bar) ** 2
            anova_table['SS']['Between Groups'] = SSTR['steps'].sum()

            # calculate SSE and update anova table
            SSE = (fitbit_data.groupby('Week').count() - 1) * fitbit_data.groupby('Week').std() ** 2
            anova_table['SS']['Within Groups'] = SSE['steps'].sum()

            # calculate SSTR and update anova table
            SSTR = SSTR['steps'].sum() + SSE['steps'].sum()
            anova_table['SS']['Total'] = SSTR

            # update degree of freedom
            anova_table['df']['Between Groups'] = fitbit_data['Week'].nunique() - 1
            anova_table['df']['Within Groups'] = fitbit_data.shape[0] - fitbit_data['Week'].nunique()
            anova_table['df']['Total'] = fitbit_data.shape[0] - 1

            # calculate MS
            anova_table['MS'] = anova_table['SS'] / anova_table['df']

            # calculate F
            F = anova_table['MS']['Between Groups'] / anova_table['MS']['Within Groups']
            anova_table['F']['Between Groups'] = F

            # p-value
            anova_table['P-value']['Between Groups'] = 1 - stats.f.cdf(F, anova_table['df']['Between Groups'],
                                                                      anova_table['df']['Within Groups'])

            # F critical
            alpha = 0.05
            # possible types "right-tailed, left-tailed, two-tailed"
            tail_hypothesis_type = "two-tailed"
            if tail_hypothesis_type == "two-tailed":
                alpha /= 2
            anova_table['F crit']['Between Groups'] = stats.f.ppf(1 - alpha, anova_table['df']['Between Groups'],
                                                                  anova_table['df']['Within Groups'])

            # Final ANOVA Table
            print(anova_table)
            anova_table.to_excel(writer, sheet_name=files[0:-4])

writer.close()

PREPILOT

6058391a4cbd8a20b68b237c
                      SS df   MS    F P-value F crit
Source of Variation                                 
Between Groups       0.0  0  NaN  NaN     NaN    NaN
Within Groups        0.0  0  NaN                    
Total                0.0  0  NaN                    
60584c5f4cbd8a20b68b237e
                                  SS     df            MS          F P-value  \
Source of Variation                                                            
Between Groups          365621.56911     22  16619.162232  19.073968     0.0   
Within Groups        33105942.720324  37996    871.300735                      
Total                33471564.289434  38018    880.413601                      

                      F crit  
Source of Variation           
Between Groups       1.67222  
Within Groups                 
Total                         
605987294cbd8a20b68b2380
                                 SS    df           MS         F   P-value  \
Source of Variat